In [1]:
#Importing Libraries
#Data Handling
import pandas as pd
import numpy as np
import sklearn
import math

import sys
sys.path.append('Write here the directory path of Code Supplementary Materil folder')

#Pipelines
from source import *
import source.pipes as op

#Sampling
from sklearn.model_selection import train_test_split

#Storing trains
import pickle
%load_ext autoreload
%autoreload 2

In [2]:
ds_train = pd.read_csv('../../data/to-ml-adult/adult.data', sep = ',', na_values =' ?').fillna('unkown')
ds_test = pd.read_csv('../../data/to-ml-adult/adult.test', na_values =' ?').fillna('unkown')

ds_final = pd.concat([ds_train, ds_test], ignore_index=True)

In [16]:
for i, x in enumerate(ds_final.columns):
    print("{0}-{1}: {2}, Nan: {3:2f}".format(i, x, ds_final[x].dtype, 100*ds_final[x].isna().sum()/len(ds_final)))

0-age: int64, Nan: 0.000000
1-workclass: object, Nan: 0.000000
2-fnlwgt: int64, Nan: 0.000000
3-education: object, Nan: 0.000000
4-education-num: int64, Nan: 0.000000
5-marital-status: object, Nan: 0.000000
6-occupation: object, Nan: 0.000000
7-relationship: object, Nan: 0.000000
8-race: object, Nan: 0.000000
9-sex: object, Nan: 0.000000
10-capital-gain: int64, Nan: 0.000000
11-capital-loss: int64, Nan: 0.000000
12-hours-per-week: int64, Nan: 0.000000
13-native-country: object, Nan: 0.000000
14-income: object, Nan: 0.000000


In [17]:
ds_final.iloc[32560,[14]].values

array([' >50K'], dtype=object)

In [47]:
#Selecting useful variables
Y = ds_final.iloc[:,[14]]
y = pd.DataFrame([1 if '>50K' in i else 0 for i in Y['income']])
A = ds_final.iloc[:,[9]]
X = ds_final.iloc[:,[0, 1, 4, 5, 6, 7, 8, 10, 11, 12, 13]]


# Split Dataset

Split dataset into train and test. Train will be used to fit and transform estimators, such as imputation, normalization, and predictive models. Notice that estimators will be used for transform in the data test only.

For this case, 80% and 20% will be used for train and test respectively.

In [48]:
causal_context = (y, pd.concat([X,A], axis=1, ignore_index=False))

In [49]:
_, X_A = causal_context
X_A.dtypes

age                int64
workclass         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
sex               object
dtype: object

In [42]:
for i, x in enumerate(X.columns):
    print("{0}-{1}: {2}, Nan: {3:2f}".format(i, x, X[x].dtype, 100*X[x].isna().sum()/len(X)))

0-age: int64, Nan: 0.000000
1-workclass: object, Nan: 0.000000
2-education-num: int64, Nan: 0.000000
3-marital-status: object, Nan: 0.000000
4-occupation: object, Nan: 0.000000
5-relationship: object, Nan: 0.000000
6-race: object, Nan: 0.000000
7-capital-gain: int64, Nan: 0.000000
8-capital-loss: int64, Nan: 0.000000
9-hours-per-week: int64, Nan: 0.000000
10-native-country: object, Nan: 0.000000
11-sex: object, Nan: 0.000000


In [53]:
analysis_scenarios = ['adult', 'fairness']

for key in analysis_scenarios:
    print('Considering {} context'.format(key))
    Y_final, X_final = causal_context
    
    X_train, X_test, y_train, y_test = train_test_split(X_final, Y_final, test_size = 0.3, stratify= Y_final, random_state = 1)
    A_train = X_train.loc[:,['sex']]
    A_test = X_test.loc[:,['sex']]

    #Get the datasets considering X_test, y_test, and A_test
    X_train_notaware_sex = X_train.iloc[:,:-1]
    X_train_aware = X_train
    
    #Creating a dictionary for datasets
    train_datasets = {}
    train_datasets = {
        'aware': X_train_aware,
        'sex': X_train_notaware_sex
    }
    

    #Dictionary where pickles will be stored
    preprocessing_pickles = {}
    preprocessing_pickles_norm = {}
    
    enc = 'label'
    
    idnumerical = [0, 2, 7, 8, 9]
    
    #Creating pickles for each case
    for td in train_datasets:
        print('Training prepro for {}'.format(td))
        data = train_datasets[td]
        X, pipe_nom, pipe_num, numerical_features , nominal_features = op.preprocessing(data, idnumerical=idnumerical, imputation=True, encode = enc, normalization = False)
        preprocessing_pickles[td] = (X, pipe_nom, pipe_num)
        
        X, pipe_nom, pipe_num, numerical_features , nominal_features = op.preprocessing(data, idnumerical=idnumerical, imputation=True, encode = enc, normalization = True)
        preprocessing_pickles_norm[td] = (X, pipe_nom, pipe_num)
        
        numerical_features.extend(nominal_features)        
        with open('../../pipes/adult-income/'+key+'/preprocessing_features_'+td+'.pickle', 'wb') as f:
            pickle.dump(numerical_features, f)
            
        print('Total of variables: {0}'.format(len(numerical_features)))

    #Run following codes for storing pipelines on pickles
    with open('../../pipes/adult-income/'+key+'/preprocessing.pickle', 'wb') as f:
        pickle.dump(preprocessing_pickles, f)
        
    with open('../../pipes/adult-income/'+key+'/preprocessing_norm.pickle', 'wb') as f:
        pickle.dump(preprocessing_pickles_norm, f)
    
    #Get the datasets considering X_test, y_test, and A_test
    X_test_notaware_sex= X_test.iloc[:,:-1]
    X_test_aware = X_test
    
    test_datasets = {}
    test_datasets = {
    'aware': X_test_aware,
    'sex': X_test_notaware_sex,
    }
    
    #Applying Preprocessing Pipelines
    data_test_prepro = {}
    data_test_prepro_norm = {}
    
    for d in preprocessing_pickles.keys():
        print("Transforming test dataset {}".format(d))
        ds = test_datasets[d]
        
        _ , pnom, pnum = preprocessing_pickles[d]
        prep_d = op.applypreprocessing(ds, pnom, pnum, idnumerical= idnumerical)
        data_test_prepro[d] = prep_d

        _ , pnom, pnum = preprocessing_pickles_norm[d]
        prep_d = op.applypreprocessing(ds, pnom, pnum, idnumerical=idnumerical)
        data_test_prepro_norm[d] = prep_d
        
    
    with open('../../pipes/adult-income/'+key+'/preprocessing_test.pickle','wb') as f:
        pickle.dump(data_test_prepro,f)
    with open('../../pipes/adult-income/'+key+'/preprocessing_test_norm.pickle', 'wb') as f:
        pickle.dump(data_test_prepro_norm, f)
    
    
    y = {'train': y_train,
         'test': y_test}
    
    with open('../../pipes/adult-income/'+key+'/y.pickle', 'wb') as f:
        pickle.dump(y,f)
    
    A = {'train': A_train,
         'test': A_test}
    with open('../../pipes/adult-income/'+key+'/A.pickle','wb') as f:
        pickle.dump(A, f)
        
    print()

Considering adult context
Training prepro for aware
Total of variables: 12
Training prepro for sex
Total of variables: 11
Transforming test dataset aware
Transforming test dataset sex

Considering fairness context
Training prepro for aware
Total of variables: 12
Training prepro for sex
Total of variables: 11
Transforming test dataset aware
Transforming test dataset sex

